In [3]:
%matplotlib inline

import os, random, cv2, glob, itertools, shutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from numpy.random import multinomial
from natsort import natsorted

from PIL import Image 

import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
from torch.utils.data import DataLoader, random_split
from torch.autograd import Variable
from torch.profiler import profile, record_function, ProfilerActivity
import torchvision
from torchvision import datasets, transforms, utils
from tqdm.notebook import tqdm

import warnings
warnings.filterwarnings('ignore')


from obspy import read, Trace, Stream, UTCDateTime
from obspy.core import AttribDict
from obspy.io.segy.segy import SEGYTraceHeader, SEGYBinaryFileHeader
from obspy.io.segy.core import _read_segy
from obspy.io.segy.segy import iread_segy

import matplotlib.cm as cm
from mpl_toolkits.mplot3d import Axes3D

from dataset import DatasetFromFolderPy
from NDMnet import Generator

from numpy import linalg as LA
from natsort import natsorted


In [4]:
DIR_RESULTS = 'results/'
DIR_TRAINING_DATASET_X = 'DataFft/Model5M/mar5z_5/'
DIR_TRAINING_DATASET_Y = 'DataFft/Model1M/mar1z_125/'
epoch = 700

alpha = 10e-9 #Normalization coefficient

def read_data(pathA, pathB):
    dataA = glob.glob(pathA+'*.npy')
    dataB = glob.glob(pathB + '*.npy')
    dataA = natsorted(dataA)
    dataB = natsorted(dataB)
    print(len(dataA))

    data = [list(tup) for tup in zip(dataA,dataB)]
    return data

data = read_data(DIR_TRAINING_DATASET_X,DIR_TRAINING_DATASET_Y)

data = natsorted(data)
print(data)
print(len(data))

0
[]
0


In [ ]:
test_data_idx = np.arange(171)
train_data_idx = np.load(DIR_RESULTS+'jdx60.npy')
test_data_idx = np.delete(test_data_idx, train_data_idx)

ngf = 64
G = Generator(4, ngf, 4)
G.cuda()
G.normal_weight_init(mean=0.0, std=0.02)
G.load_state_dict(torch.load(DIR_RESULTS+'G_'+str(epoch)+'.pkl'))
    
L2_Gy = []
L2_relative = []
for k in test_data_idx:
    fft_x_cr = np.load(data[k][0])
    x_real = fft_x_cr.real
    x_imag = fft_x_cr.imag
    x_real = x_real / alpha
    x_imag = x_imag/ alpha
    x = np.stack([x_real,x_imag])
    x = torch.from_numpy(x)
    x = x.type(torch.FloatTensor)
    x = Variable(x.cuda())
    x = x[None,:]

    gen_image = G(x) 
    gen_image = gen_image.cpu().data
    gen_image = gen_image * alpha
    fft_gen = torch.complex(gen_image[0][0], gen_image[0][1])
        
    seism_fft_positive_for_lr = fft_gen[:,1:]
    seism_fft_positive = fft_gen
    seism_fft_positive = torch.tensor(seism_fft_positive)
    seism_fft_positive_for_lr = torch.tensor(seism_fft_positive_for_lr)
    tmp_zeros = torch.zeros(512,2500-512-511)
    seism_fft_new_real = torch.cat(((seism_fft_positive.real), tmp_zeros, torch.fliplr(seism_fft_positive_for_lr.real) ),1)
    seism_fft_imag = torch.cat( (seism_fft_positive.imag, tmp_zeros, torch.fliplr(-(seism_fft_positive_for_lr.imag)) ), 1 )
    seism_fft_new = torch.complex(seism_fft_new_real, seism_fft_imag)
    seism_new_new_Gy = torch.fft.ifft2(seism_fft_new,dim=1)
    

    fft_y_cr = np.load(data[k][1])                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              
    y_real = fft_y_cr.real
    y_imag = fft_y_cr.imag
    y = np.stack([y_real,y_imag])                                                                                                        
    y = torch.from_numpy(y)
    y = y[None,:]
        
    
    fft_y = torch.complex(y[0][0], y[0][1])
    seism_fft_positive_for_lr = fft_y[:,1:]
    seism_fft_positive = fft_y
    seism_fft_positive = torch.tensor(seism_fft_positive)
    seism_fft_positive_for_lr = torch.tensor(seism_fft_positive_for_lr)
    tmp_zeros = torch.zeros(512,2500-512-511)
    seism_fft_new_real = torch.cat(((seism_fft_positive.real), tmp_zeros, torch.fliplr(seism_fft_positive_for_lr.real) ),1)
    seism_fft_imag = torch.cat( (seism_fft_positive.imag, tmp_zeros, torch.fliplr(-(seism_fft_positive_for_lr.imag)) ), 1 )
    seism_fft_new = torch.complex(seism_fft_new_real, seism_fft_imag)
    seism_new_new_y = torch.fft.ifft2(seism_fft_new,dim=1)
      
        
    res = 2* LA.norm(seism_new_new_y-seism_new_new_Gy) / (LA.norm(seism_new_new_y)+LA.norm(seism_new_new_Gy))        
    L2_Gy.append(res)
    res = 100*LA.norm(seism_new_new_y - seism_new_new_Gy) / LA.norm(seism_new_new_y)
    L2_relative.append(res)
            
    np.save(DIR_RESULTS+'seism' + str(epoch)+ '_L2_Gy', L2_Gy)
    np.save(DIR_RESULTS+'seism' + str(epoch)+ '_L2_rel_Gy', L2_relative)

In [ ]:
test_data_idx = np.arange(171)
train_data_idx = np.load(DIR_RESULTS+'jdx60.npy')
test_data_idx = np.delete(test_data_idx, train_data_idx)

L2_xy = []
for k in test_data_idx:
    fft_x_cr = np.load(data[k][0])
    x_real = fft_x_cr.real
    x_imag = fft_x_cr.imag
    x = np.stack([x_real,x_imag])
    x = torch.from_numpy(x)
    x = x[None,:]
    
    fft_x = torch.complex(x[0][0], x[0][1])
    seism_fft_positive_for_lr = fft_x[:,1:]
    seism_fft_positive = fft_x
    seism_fft_positive = torch.tensor(seism_fft_positive)
    seism_fft_positive_for_lr = torch.tensor(seism_fft_positive_for_lr)


    tmp_zeros = torch.zeros(512,2500-512-511)
    seism_fft_new_real = torch.cat(((seism_fft_positive.real), tmp_zeros, torch.fliplr(seism_fft_positive_for_lr.real) ),1)
    seism_fft_imag = torch.cat( (seism_fft_positive.imag, tmp_zeros, torch.fliplr(-(seism_fft_positive_for_lr.imag)) ), 1 )

    seism_fft_new = torch.complex(seism_fft_new_real, seism_fft_imag)
    seism_new_new_x = torch.fft.ifft2(seism_fft_new,dim=1)
    
    fft_y_cr = np.load(data[k][1])                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              
    y_real = fft_y_cr.real
    y_imag = fft_y_cr.imag
    y = np.stack([y_real,y_imag])                                                                                                        
    y = torch.from_numpy(y)
    y = y[None,:]
    
    fft_x = torch.complex(y[0][0], y[0][1])
    seism_fft_positive_for_lr = fft_x[:,1:]
    seism_fft_positive = fft_x
    seism_fft_positive = torch.tensor(seism_fft_positive)
    seism_fft_positive_for_lr = torch.tensor(seism_fft_positive_for_lr)


    tmp_zeros = torch.zeros(512,2500-512-511)
    seism_fft_new_real = torch.cat(((seism_fft_positive.real), tmp_zeros, torch.fliplr(seism_fft_positive_for_lr.real) ),1)
    seism_fft_imag = torch.cat( (seism_fft_positive.imag, tmp_zeros, torch.fliplr(-(seism_fft_positive_for_lr.imag)) ), 1 )

    seism_fft_new = torch.complex(seism_fft_new_real, seism_fft_imag)
    seism_new_new_y = torch.fft.ifft2(seism_fft_new,dim=1)
      
                                    
    res = 100* LA.norm(seism_new_new_y-seism_new_new_x) / (LA.norm(seism_new_new_y))      
      
    L2_xy.append(res)
        
np.save(DIR_RESULTS+'init_L2_relative_xy_seism', L2_xy)


In [ ]:
print('L2(x,y)',np.mean(L2_xy))
print('L2(G(x),y)',np.mean(L2_relative))